In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import string
import numpy as np

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from math import log

In [2]:
kata=[]
dictio = {}

In [3]:
def bersihkan_inputan(eachLine):
    """
    case folding
    """
    eachLine = eachLine.lower()
    eachLine = re.sub(r"\d+", "", eachLine)
    eachLine = eachLine.translate(str.maketrans("","",string.punctuation))
    #hapus simbol
    eachLine = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',eachLine)
    eachLine = re.sub('rt @[^\s]+',' ', eachLine)
    eachLine = re.sub('@[^\s]+',' ',eachLine)
    eachLine = re.sub(r'[^\x00-\x7F]+',' ', eachLine)
    #Remove additional white spaces
    eachLine = re.sub('[\s]+', ' ', eachLine)
    eachLine = re.sub(r'#([^\s]+)', r'\1', eachLine)
    eachLine = eachLine.strip()
    """
    stop words
    """
    stop_words = set(stopwords.words('english')) 
  
    word_tokens = word_tokenize(eachLine) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
#   print("==========================================STEMMING")
    """
    Stemming
    """
    ps = PorterStemmer()
    for k in filtered_sentence:
        kata.append(ps.stem(k))

In [4]:
def vsm(doc, wordBank):
    for words in doc:
        for kata in wordBank:
            if words == kata:
                if words in dictio:
                    dictio[words] += 1
                else:
                    dictio[words] = 1

In [6]:
def idf(doc, wordBank):
    #hitung jumlah doc
    N = 3
    #buat dataframe dengan header word dan idf
    result = pd.DataFrame(columns=['word_bank', 'df','idf','vsm'])
#     #untuk setiap kata pada wordBank lakukan.....
    for word in doc:
#         print(word)
        jumlah = 0
        for kata in wordBank:
            if word == kata:
                jumlah+=1

#         #hitung inverse document frequency smooth
        idft = log(N / (jumlah + 1), 10)
        vsm = jumlah / len(doc)
#         #tambahkan idf untuk setiap kata pada data frame
        result = result.append(pd.Series([word,  jumlah, idft, vsm], index=['word_bank', 'df','idf','vsm']), ignore_index=True)
#     #return variable result
    return result
#     print(result)

In [7]:
df=pd.read_csv("hasil-baru.csv")
queryFile = "query"
QueryLines = [line.rstrip('\n') for line in open(queryFile)]
#     print(QueryLines)
for eachLine in QueryLines:
    bersihkan_inputan(eachLine)
dfidf=idf(kata, df['Term / kata'])
vsm(kata, df['Term / kata'])

In [10]:
dictio

{'branson': 3,
 'show': 8,
 'us': 2,
 'screen': 1,
 'sir': 4,
 'richard': 4,
 'tv': 1,
 'prove': 1,
 'rate': 2,
 'programm': 1,
 'suffer': 1,
 'poor': 1,
 'view': 1,
 'figur': 1,
 'debut': 1,
 'fox': 3,
 'network': 2,
 'three': 1,
 'week': 1,
 'ago': 1,
 'lost': 1,
 'one': 1,
 'five': 2,
 'viewer': 3,
 'sinc': 1,
 'seen': 2,
 'answer': 1,
 'featur': 1,
 'hit': 1,
 'rival': 1,
 'see': 1,
 'group': 1,
 'young': 1,
 'compet': 1,
 'becom': 1,
 'presid': 1,
 'busi': 2,
 'empir': 1,
 'averag': 1,
 'million': 2,
 'st': 1,
 'rank': 2,
 'primetim': 2,
 'alreadi': 1,
 'complet': 1,
 'first': 1,
 'season': 1,
 'still': 1,
 'manag': 1,
 'pull': 1,
 'number': 1,
 'four': 1,
 'said': 2,
 'plan': 1,
 'stick': 1,
 'throughout': 1,
 'episod': 1,
 'run': 1,
 'go': 1,
 'remain': 1,
 'air': 1,
 'spokesman': 1,
 'scott': 1,
 'creativ': 1,
 'extrem': 1,
 'pleas': 1,
 'hope': 1,
 'audienc': 1,
 'find': 1,
 'ad': 1,
 'contest': 1,
 'confront': 1,
 'task': 1,
 'solv': 1,
 'similar': 1,
 'also': 1,
 'subject': 

In [6]:
dfidf

,word_bank,df,idf,vsm
0,branson,1,0.176091,0.007143
1,show,1,0.176091,0.007143
2,flop,0,0.477121,0.000000
3,us,1,0.176091,0.007143
4,screen,1,0.176091,0.007143
...,...,...,...,...
135,jump,1,0.176091,0.007143
136,taxi,0,0.477121,0.000000
137,trump,0,0.477121,0.000000
138,seen,1,0.176091,0.007143


In [9]:
dfidf[dfidf['word_bank'].str.contains("show")]
# for kata in dfidf['word_bank']:
#     print(kata)

,word_bank,df,idf,vsm
1,show,1,0.176091,0.007143
11,show,1,0.176091,0.007143
33,show,1,0.176091,0.007143
48,show,1,0.176091,0.007143
67,show,1,0.176091,0.007143
87,show,1,0.176091,0.007143
102,show,1,0.176091,0.007143
110,show,1,0.176091,0.007143
134,shown,1,0.176091,0.007143


In [9]:
a = df[df['text'].str.contains("tv")]

In [10]:
a['text']

72     According to London author Yasmin Khan (“The S...
77     Michelin season has arrived in San Francisco a...
98     It has been a pivotal year for both the restau...
129    It has never been easier — or more delicious —...
Name: text, dtype: object

In [15]:
int(df[df['Term / kata'] == "us"]['business'] + df[df['Term / kata'] == "us"]['entertainment'])

104

In [3]:
di = {}
di['aku'] = 1
di['aku'] += 1
if 'akus' in di:
    print(True)